In [2]:
import pandas as pd
import numpy as np
import re

# Read original rFBA model

PDF file needs to be parsed manually...

(in order to be in accordance with the syntax of 'id' attribute values of the SBML type 'SId' (Reference: L3V2 Section 3.1.7), the metabolites 13PDG, 2PG, and 3PG were renamed to PDG13, PG2, and PG3, respectively.)

In [3]:
#import PyPDF2
#pdfFileObj = open('Covert2002/SI.pdf', 'rb')
#pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#pageObj = pdfReader.getPage(3)
#pageObj.extractText()

In [4]:
# reactions
SI1 = pd.read_csv('SI-1.tsv', sep='\t')

In [5]:
# metabolites
SI2 = pd.read_csv('SI-2.tsv', sep='\t')
SI2 = SI2.sort_values(by=['id'])

## Add Amino Acid-producing enzyme and Ribosome

In [6]:
SI1 = SI1.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Gene': 'aspA', 'Reaction': 'FUM <-> Z'}, ignore_index=True)
SI1 = SI1.append({'ID': 'R', 'Protein': 'Ribosome', 'Gene': 'rplA, rplB, rplC, rplD, rplE, rplF, rplI, rplJ, rplK, rplL, rplM, rplN, rplO, rplP, rplQ, rplR, rplS, rplT, rplU, rplV, rplW, rplX, rplY, rpmA, rpmB, rpmC, rpmD, rpmE, rpmF, rpmG, rpmH, rpmI, rpmJ, rpsA, rpsB, rpsC, rpsD, rpsE, rpsF, rpsG, rpsH, rpsI, rpsJ, rpsK, rpsL, rpsM, rpsN, rpsO, rpsP, rpsQ, rpsR, rpsS, rpsT, rpsU'}, ignore_index=True)

## Reformulate gene IDs

In [7]:
# Reformulation of "lumped" IDs
SI1.loc[SI1.ID == 'ACEE', 'Gene'] = 'aceE, aceF, lpdA'
SI1.loc[SI1.ID == 'ATPAR', 'Gene'] = 'atpA, atpB, atpC, atpD, atpE, atpF, atpG, atpH, atpI'
SI1.loc[SI1.ID == 'CYDA', 'Gene'] = 'cydA, cydB'
SI1.loc[SI1.ID == 'CYOA', 'Gene'] = 'cyoA, cyoB, cyoC, cyoD'
SI1.loc[SI1.ID == 'FDNG', 'Gene'] = 'fdnG, fdnH, fdnI'
SI1.loc[SI1.ID == 'FDOH', 'Gene'] = 'fdoI, fdoH, fdoG'
SI1.loc[SI1.ID == 'FRDA', 'Gene'] = 'frdA, frdB, frdC, frdD'
SI1.loc[SI1.ID == 'GLPA', 'Gene'] = 'glpA, glpB, glpC'
SI1.loc[SI1.ID == 'NUOA', 'Gene'] = 'nuoA, nuoB, nuoE, nuoF, nuoG, nuoH, nuoI, nuoJ, nuoK, nuoL, nuoM, nuoN'
SI1.loc[SI1.ID == 'PFLA', 'Gene'] = 'pflA, pflB'
SI1.loc[SI1.ID == 'PFLC', 'Gene'] = 'pflC, pflD'
SI1.loc[SI1.ID == 'PNTA1', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'PNTA2', 'Gene'] = 'pntA, pntB'
SI1.loc[SI1.ID == 'SDHA1', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SDHA2', 'Gene'] = 'sdhA, sdhB, sdhC, sdhD'
SI1.loc[SI1.ID == 'SUCA', 'Gene'] = 'sucA, sucB, lpdA'
SI1.loc[SI1.ID == 'SUCCR', 'Gene'] = 'sucC, sucD'
SI1.loc[SI1.ID == 'GLCPTS', 'Gene'] = 'ptsG, ptsH, ptsI, crr'
SI1.loc[SI1.ID == 'PIUP2R', 'Gene'] = 'pitA, pitB'
SI1.loc[SI1.ID == 'RIBUPR', 'Gene'] = 'rbsA, rbsB, rbsC, rbsD'

In [8]:
# manual curation
SI1.loc[SI1.Protein == 'Catabolite activator protein', 'Gene'] = 'cra' # 'cra (fruR)' - beide Namen bezeichnen das gleiche Gen
SI1.loc[SI1.ID == 'GLCUP', 'Gene'] = 'galP' # 'galP, etc.' - galP ist ausreichend
SI1.loc[SI1.ID == 'PCKA', 'Gene'] = 'pck' # 'pckA'
SI1.loc[SI1.ID == 'SFCA', 'Gene'] = 'maeA' # 'sfcA'
SI1.loc[SI1.ID == 'GPMBR', 'Gene'] = 'gpmA' # 'sfcA' - gpmB gibt es nicht; gpmA katalysiert die gleiche Reaktion

In [9]:
genes_list = []

for index, value in SI1['Gene'].items():
    if isinstance(value, str):
        for g in value.split(', '):
            if g not in genes_list:
                genes_list.append(g)

# Species

In [10]:
# Split SI2 into intra- and extracellular metabolites

# add column with boolean variable
SI2['external'] = 1
SI2['external'] = SI2['name'].str.contains("External")

# split dataset
extracellular = SI2.query("external")
intracellular = SI2.query("external == False")

## Intracellular Metabolites $\mathcal{X}$

In [11]:
# drop boolean column
intracellular = intracellular.drop(['external'], axis=1)

In [12]:
intracellular

,id,name
12,AC,Acetate
16,ACCOA,Acetyl-CoA
20,ACTP,Acetyl-phosphate
28,ADP,Adenosine diphosphate
32,AKG,a-Ketoglutarate
...,...,...
59,UDPGAL,UDP Galactose
63,UTP,Uridine triphosphate
67,X5P,Xylulose-5-phosphate
44,bDGLAC,b-D-Galactose


In [13]:
# add amino acid
intracellular = intracellular.append({'id': 'Z', 'name': 'amino acid'}, ignore_index=True)

In [14]:
# add attributes
intracellular["compartment"] = 'cytosol'
intracellular["constant"] = 'false'
intracellular["boundaryCondition"] = 'false'
intracellular["hasOnlySubstanceUnits"] = 'true'
intracellular["ram:speciesType"] = 'metabolite'

## Extracellular Metabolites $\mathcal{Y}$

In [15]:
# drop boolean column
extracellular = extracellular.drop(['external'], axis=1)

In [16]:
# abundant species
extracellular.at[extracellular['id'] == 'CO2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'HEXT', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'PIxt', 'initialAmount'] = 10000

# potential product are initially zero
extracellular.at[extracellular['id'] == 'ETHxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'FORxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'LCTSxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'PYRxt', 'initialAmount'] = 0

# carbon sources and O2: initial amount depends on the scenario; aerobic glucose as default:
extracellular.at[extracellular['id'] == 'ACxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'GLCxt', 'initialAmount'] = 100
extracellular.at[extracellular['id'] == 'GLxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'O2xt', 'initialAmount'] = 10000
extracellular.at[extracellular['id'] == 'RIBxt', 'initialAmount'] = 0
extracellular.at[extracellular['id'] == 'SUCCxt', 'initialAmount'] = 0

In [17]:
# add attributes
extracellular["compartment"] = 'extracellular'
extracellular["constant"] = 'false'
extracellular["boundaryCondition"] = 'false' # not false for every species
extracellular["hasOnlySubstanceUnits"] = 'true'
extracellular["ram:speciesType"] = 'extracellular'

## Macromolecules $\mathcal{P}$

### Enzymes $\mathcal{E}$

In [18]:
# create copy of first part of SI
enzymes = SI1[['ID', 'Protein', 'Gene']].copy()

In [19]:
# delete rows without ID (i.e. regulatory proteins), without Gene (i.e. spontaneous) or without Protein (exchange, biomass, maintenance)
enzymes.dropna(subset=['ID', 'Protein', 'Gene'], inplace=True) # in place: Keep df with valid entries in the same variable

In [20]:
# create enzyme IDs ('E_' + reaction id)
enzymes["id"] = 'E_' + enzymes["ID"]
enzymes = enzymes.drop(['ID'], axis=1) # delete old IDs
enzymes = enzymes[['id', 'Protein', 'Gene']] # order columns

In [21]:
enzymes = enzymes.rename(columns={"Protein": "name"})

In [22]:
# add attributes
enzymes["compartment"] = 'cytosol'
enzymes["constant"] = 'false'
enzymes["boundaryCondition"] = 'false'
enzymes["hasOnlySubstanceUnits"] = 'true'
enzymes["initialAmount"] = np.nan
enzymes["ram:molecularWeight"] = None # are set later
enzymes["ram:objectiveWeight"] = None # are set later
enzymes["ram:biomassPercentage"] = 0.0
enzymes["ram:speciesType"] = 'enzyme'
enzymes["degradationRate"] = 0.01

In [23]:
enzymes.at[enzymes["id"] == "R", "degradationRate"] = 0.001

### Regulatory Proteins $\mathcal{RP}$

In [24]:
# create copy of first part of SI
rp = SI1[['ID', 'Protein', 'Gene']].copy()

In [25]:
# keep rows without ID
rp = rp[pd.isnull(rp['ID'])]
rp = rp.drop(['ID'], axis=1)

In [26]:
# rp id = 'RP_' + gene id
rp["id"] = 'RP_' + rp["Gene"]
rp = rp[['id', 'Protein', 'Gene']] # order columns

In [27]:
rp = rp.rename(columns={"Protein": "name"})

In [28]:
# add attributes
rp["compartment"] = 'cytosol'
rp["constant"] = 'false'
rp["boundaryCondition"] = 'false'
rp["hasOnlySubstanceUnits"] = 'true'
rp["initialAmount"] = np.nan
rp["ram:biomassPercentage"] = 0.0
rp["ram:molecularWeight"] = None # are set later
rp["ram:objectiveWeight"] = None # are set later
rp["ram:speciesType"] = 'enzyme'
rp["degradationRate"] = 0.2

### Quota $\mathcal{Q}$

In [30]:
quota = SI1[['Reaction']].loc[SI1['Protein'] == "Biomass production flux"]

In [31]:
biomass_rxn = quota['Reaction'].values

In [32]:
stoich = []
spec = []
product = False

for s in biomass_rxn[0].split():
    if '-' in s:
        product = True
    try:
        s = float(s)
        if product:
            stoich.append(s)
        else:
            stoich.append(-s)
    except ValueError:
        if '+' not in s and '-' not in s:
            spec.append(s)

In [33]:
mw_mol = [503.15, 662.42, 741.38, 258.12, 258.12, 228.09, 198.07, 170.06, 183.03, 165.02, 87.05, 805.54, 130.06, 144.08, 763.50, 424.18, 94.98, 663.42, 740.38, 1000]
mw_mmol = [mw/1000 for mw in mw_mol]

In [34]:
biomass = pd.DataFrame({'id': spec, 'stoichiometry': stoich, 'molecular weight [g/mmol]': mw_mmol})

In [35]:
biomass['weight [g]'] = biomass['stoichiometry'] * biomass['molecular weight [g/mmol]']

In [36]:
# calculate weight of biomass
educt_surplus = biomass['weight [g]'].sum() - 1.0 # subtract biomass 'weight'
biomass_weight = -educt_surplus # new biomass weight equals educt surplus

In [37]:
# set weight of 'biomass', which will be reformulated as Quota_rest
biomass.at[biomass['id'] == 'Biomass', 'biomass_weight'] = biomass_weight

Fraction of proteins $\varphi_{\mathrm{protein}}$ is set to 0.7 (for details, see report), so the 'remaining' quota will make up 30 %

In [38]:
# add attributes for quota 'rest'
quota["ID"] = 'Quota_R'
quota["ram:molecularWeight"] = biomass_weight
quota["ram:objectiveWeight"] = biomass_weight
quota["ram:biomassPercentage"] = 0.3
quota["degradationRate"] = 0.001

#### adjust protein fraction $\phi^*_{\text{protein}}$
Determine fraction of proteins explicitly modeled using data by *Li et al.* for E. coli on minimal medium

In [39]:
li = pd.read_csv('Li_minimal.tsv', sep='\t')

In [40]:
# add column specifying whether the protein is included in the model
li['included'] = 1
li['included'] = li['!Protein:Name'].isin(genes_list)

In [41]:
li_included = li.query("included")

In [42]:
n_missing_genes = len(genes_list)-len(li_included)
frac_explicit = li_included['!Abundance:[original]'].sum()/li['!Abundance:[original]'].sum()
frac_adjusted = 0.55*(1-li_included['!Abundance:[original]'].sum()/li['!Abundance:[original]'].sum())

In [43]:
print("%i genes from the model are not included in the dataset by Li et al." %n_missing_genes)

13 genes from the model are not included in the dataset by Li et al.


In [44]:
print("The fraction of protein explicitly modeled in the dataset by Li et al. is %f%%. Therefore, the adjusted protein quota is %f%% and the fraction of explicitly modeled proteins in the model is %f%%" %(frac_explicit*100, frac_adjusted*100, (0.55-frac_adjusted)*100))

The fraction of protein explicitly modeled in the dataset by Li et al. is 30.310474%. Therefore, the adjusted protein quota is 38.329239% and the fraction of explicitly modeled proteins in the model is 16.670761%


In [45]:
# add protein quota
# weights are determined later: length will be set to median length of all proteins explicitly modeled
quota = quota.append({"ID": 'Quota_protein', "Gene": 'R', "ram:biomassPercentage": frac_adjusted, "degradationRate": 0.01}, ignore_index=True)

In [46]:
quota['compartment'] = 'cytosol'
quota["initialAmount"] = np.nan

In [47]:
quota

,Reaction,ID,ram:molecularWeight,ram:objectiveWeight,ram:biomassPercentage,degradationRate,Gene,compartment,initialAmount
0,41.25 ATP + 3.54 NAD + 18.22 NADPH + 0.2 G6P +...,Quota_R,0.864801,0.864801,0.300000,0.001,NaN,cytosol,NaN
1,NaN,Quota_protein,NaN,NaN,0.383292,0.010,R,cytosol,NaN


# Reactions $\mathcal{R}$

## Metabolic Reactions $\mathcal{R_X}$ and Uptake Reactions $\mathcal{R_Y}$

In [48]:
# create copy of first part of SI
reactions = SI1[['ID', 'Protein', 'Gene', 'Reaction']].copy()

In [49]:
# delete rows without Reaction
reactions.dropna(subset=['Reaction'], inplace=True)

In [50]:
# delete exchange reactions
reactions = reactions[~reactions.Protein.str.contains('exchange')]

In [51]:
# delete maintenance and biomass reactions
reactions.drop(reactions[reactions.ID == 'ATPM'].index, inplace=True)
reactions.drop(reactions[reactions.ID == 'VGRO'].index, inplace=True)

In [52]:
# add reaction for amino acid
reactions = reactions.append({'ID': 'ASPT', 'Protein': 'L-aspartase', 'Reaction': 'FUM <-> Z'}, ignore_index=True)

### turnover rates

In [53]:
reactions_turnover = pd.read_csv('TurnoverRates.tsv', sep='\t')

In [54]:
# add column with boolean variable
reactions_turnover['transport'] = 1
reactions_turnover['transport'] = reactions_turnover['Protein'].str.contains("transport|efflux|uptake|secretion|permease")

# split dataset
turnover_transport = reactions_turnover.query("transport")
turnover_metabolic = reactions_turnover.query("transport == False")

In [55]:
missing = reactions_turnover['kcat_fwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_bwd/s'].str.count('kcat').sum()+reactions_turnover['kcat_fwd/s'].str.count('diffusion').sum()+reactions_turnover['kcat_bwd/s'].str.count('diffusion').sum()

In [56]:
print("Total number of reactions and reverse reactions: %i" %(len(reactions_turnover)+reactions_turnover['reversible'].sum()))
print("Missing values (total): %i (%f %%)" %(missing, missing/(len(reactions_turnover)+reactions_turnover['reversible'].sum())*100))
print("Missing values (metabolic): %i (%f %%)" %(turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()+turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_metabolic)+turnover_metabolic['reversible'].sum())*100))
print("Missing values (transport): %i (%f %%)" %(missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum(), (missing-turnover_metabolic['kcat_fwd/s'].str.count('kcat').sum()-turnover_metabolic['kcat_bwd/s'].str.count('kcat').sum())/(len(turnover_transport)+turnover_transport['reversible'].sum())*100))

Total number of reactions and reverse reactions: 152
Missing values (total): 37 (24.342105 %)
Missing values (metabolic): 7 (5.737705 %)
Missing values (transport): 30 (100.000000 %)


In [57]:
all_values = []

# Iterate over two given columns only from the dataframe
for column in reactions_turnover[['kcat_fwd/s', 'kcat_bwd/s']]:
    # Select column contents by column name using [] operator
    columnSeriesObj = reactions_turnover[column]
    for v in columnSeriesObj.values:
        try:
            v = float(v)
            if v > 0:
                all_values.append(v)
        except ValueError:
            pass

In [58]:
np.median(all_values)
#np.mean(all_values)

66.6

In [59]:
reactions_turnover = reactions_turnover.replace(to_replace = 'kcat', value = np.median(all_values))
reactions_turnover = reactions_turnover.replace(to_replace = 'diffusion', value = np.median(all_values))

In [60]:
reactions_turnover["kcat_fwd/s"] = pd.to_numeric(reactions_turnover["kcat_fwd/s"])
reactions_turnover["kcat_bwd/s"] = pd.to_numeric(reactions_turnover["kcat_bwd/s"])

In [61]:
reactions_turnover["fbc:geneProductAssociation"] = np.where(pd.isna(reactions_turnover["Gene"]), '', 'E_' + reactions_turnover["Covert ID"])
reactions_turnover['kcat_fwd/min'] = reactions_turnover['kcat_fwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover['kcat_bwd/min'] = reactions_turnover['kcat_bwd/s'].apply(lambda x: x*60) # conversion from 1/s to 1/min
reactions_turnover["fbc:maintenanceScaling"] = 0.0

## Translation $\mathcal{R_P}$

In [62]:
# Ribosomal activity: 17 amino acids / s
amino_acids_per_min = 1020
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
weight_amino_acid = 0.1331

In [63]:
translation = SI1[['ID', 'Protein', 'Gene']].copy()

In [64]:
# drop rows without Gene (spontaneous reactions...)
translation.dropna(subset=['Gene'], inplace=True)

In [65]:
translation["ID"] = np.where(pd.isna(translation["ID"]), 'T_' + translation["Gene"], 'T_' + translation["ID"])

In [66]:
translation["Length"] = None

In [67]:
uniprot = pd.read_csv('uniprot.tsv', sep='\t')

In [68]:
for index, value in translation['Gene'].items():
    length = 0
    for g in value.split(', '):
        try:
            length = length + uniprot.loc[uniprot['yourlist'] == g, 'Length'].values[0]
        except ValueError:
            pass
    translation['Length'][index] = length

In [69]:
median_length = translation['Length'].median()

In [70]:
# append translation of protein quota
translation = translation.append({"ID": 'Quota_protein', "Length": median_length}, ignore_index=True)

In [71]:
translation["reversible"] = 'false'
translation['ram:kcatForward'] = translation['Length'].apply(lambda x: amino_acids_per_min/x)
translation['ram:kcatBackward'] = 0.0
translation["fbc:maintenanceScaling"] = 0.0
translation["fbc:geneProductAssociation"] = 'R'

### set weights for macromolecules

In [72]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in enzymes['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('E_', 'T_'), 'Length'].values[0])
    enzymes.at[index, "ram:molecularWeight"] = weight
    enzymes.at[index, "ram:objectiveWeight"] = weight

In [73]:
# take molecular weight of Asp as MW for amino acid, i.e. 0.1331 g/mmol
for index, value in rp['id'].items():
    weight = weight_amino_acid * float(translation.loc[translation['ID'] == value.replace('RP_', 'T_'), 'Length'].values[0])
    rp.at[index, "ram:molecularWeight"] = weight
    rp.at[index, "ram:objectiveWeight"] = weight

In [74]:
# now set weights of protein quota
quota.at[quota['ID'] == 'Quota_protein', 'ram:molecularWeight'] = median_length * weight_amino_acid
quota.at[quota['ID'] == 'Quota_protein', 'ram:objectiveWeight'] = median_length * weight_amino_acid

## Maintenance Reaction $\mathcal{R_M}$

In [75]:
# filter maintenance reaction from SI
main_df = SI1.filter(SI1[SI1.ID == 'ATPM'].index, axis = 0)

# Regulated Proteins $p = \mathcal{RE} \cup \mathcal{RP}$

In [230]:
# create copy of first part of SI
regulated = SI1[['ID', 'Protein', 'Gene', 'Regulatory logic']].copy()

In [231]:
# delete rows without Regulation
regulated.dropna(subset=['Regulatory logic'], inplace=True)

In [232]:
# Create IDs of boolean variables representing gene expresssion for regulated proteins
# regulated enzymes: 'G_' + ID (uppercase)
# regulatory proteins: 'G_' + Gene (lowercase)
regulated["bool_ID"] = np.where(pd.isna(regulated["ID"]), 'G_' + regulated["Gene"] + '_bar', 'G_' + regulated["ID"] + '_bar')

## Events

In [76]:
events = pd.DataFrame()

### Control of boolean variables controlling RP gene expression

In [77]:
# effect of O2 only
events = events.append({'id':'low_O2', 'variable':'O2xt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_O2', 'variable':'O2xt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_arcA_bar', 'qual_con_fnr_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of SUCC only
events = events.append({'id':'low_SUCC', 'variable':'SUCCxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_SUCC', 'variable':'SUCCxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_dcuS_bar'], 'value':['1']}, ignore_index=True)
# effect of LCTS only
events = events.append({'id':'low_LCTS', 'variable':'LCTSxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['1', '1', '1']}, ignore_index=True)
events = events.append({'id':'high_LCTS', 'variable':'LCTSxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_galR_bar', 'qual_con_galS_bar', 'qual_con_lacI_bar'], 'value':['0', '0', '0']}, ignore_index=True)
# effect of GL only
events = events.append({'id':'low_GL', 'variable':'GLxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GL', 'variable':'GLxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_glpR_bar'], 'value':['0']}, ignore_index=True)
# effect of GLC only
events = events.append({'id':'low_GLC', 'variable':'GLCxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_GLC', 'variable':'GLCxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_mlc_bar'], 'value':['0']}, ignore_index=True)
# effect of RIB only
events = events.append({'id':'low_RIB', 'variable':'RIBxt', 'relation':'lt', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_RIB', 'variable':'RIBxt', 'relation':'geq', 'threshold':'theta_xt', 'assignment':['qual_con_rbsR_bar', 'qual_con_rpiR_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP DcuS only
events = events.append({'id':'low_DcuS', 'variable':'RP_dcuS', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_DcuS', 'variable':'RP_dcuS', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_dcuR_bar'], 'value':['1']}, ignore_index=True)
# effect of RP FadR only
events = events.append({'id':'low_FadR', 'variable':'RP_fadR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['0']}, ignore_index=True)
events = events.append({'id':'high_FadR', 'variable':'RP_fadR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_iclR_bar'], 'value':['1']}, ignore_index=True)

### complex regulation ###
# cra
# crp
# fadR
# pdhR

### Control of boolean variables controlling Enzyme gene expression

In [ ]:
# effect of RP IclR only
events = events.append({'id':'low_IclR', 'variable':'RP_iclR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_IclR', 'variable':'RP_iclR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEA_bar'], 'value':['0']}, ignore_index=True)
# effect of RP PdhR only
events = events.append({'id':'low_PdhR', 'variable':'RP_pdhR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['1', '1']}, ignore_index=True)
events = events.append({'id':'high_PdhR', 'variable':'RP_pdhR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_ACEE_bar', 'qual_con_SUCA_bar'], 'value':['0', '0']}, ignore_index=True)
# effect of RP FNR only
events = events.append({'id':'low_FNR', 'variable':'RP_FNR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar'], 'value':['0', '0', '1']}, ignore_index=True)
events = events.append({'id':'high_FNR', 'variable':'RP_FNR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_FDNG_bar', 'qual_con_FUMBR_bar', 'qual_con_NDH_bar'], 'value':['1', '1', '0']}, ignore_index=True)
# effect of RP ArcA only
events = events.append({'id':'low_arcA', 'variable':'RP_arcA', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_arcA', 'variable':'RP_arcA', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_MDHR_bar'], 'value':['0']}, ignore_index=True)
# effects of Cra only
events = events.append({'id':'low_Cra', 'variable':'RP_cra', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar'], 'value':['0', '1']}, ignore_index=True)
events = events.append({'id':'high_Cra', 'variable':'RP_cra', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_PPSA_bar', 'qual_con_PYKF_bar'], 'value':['1', '0']}, ignore_index=True)
# effect of RpiR only
events = events.append({'id':'low_RpiR', 'variable':'RP_rpiR', 'relation':'lt', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['1']}, ignore_index=True)
events = events.append({'id':'high_RpiR', 'variable':'RP_rpiR', 'relation':'geq', 'threshold':'theta_RP', 'assignment':['qual_con_RPIBR_bar'], 'value':['0']}, ignore_index=True)

### complex regulation ###
# ACEB
# ADHER
# CYDA
# CYOA
# FRDA
# FUMAR, SDHA1, SDHA2
# PFLA, PFLC, FORUPR, DCUC

### regulation not modeled by Covert et al. ###
# FUMCR

# Write Model as XML

In [254]:
f = open('ecoli_r-deFBA.xml', 'w').close()
f = open('ecoli_r-deFBA.xml', 'w')

## First lines

In [255]:
# XML definition
f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")

39

In [256]:
# SBML definition
f.write("<sbml xmlns=\"http://www.sbml.org/sbml/level3/version2/core\" level=\"3\" version=\"2\" xmlns:fbc=\"http://www.sbml.org/sbml/level3/version1/fbc/version2\" fbc:required=\"false\" fbc:strict=\"false\" xmlns:qual=\"http://www.sbml.org/sbml/level3/version1/qual/version1\" qual:required=\"true\">\n")

In [257]:
# Model Definition
f.write("<model id=\"ecoli_r-deFBA\" name=\"ecoli_r-deFBA\">\n\n")

## List of Compartments

In [260]:
f.write("<listOfCompartments>\n\t<compartment id=\"extracellular\" constant=\"true\"/>\n\t<compartment id=\"cytosol\" constant=\"true\"/>\n</listOfCompartments>\n\n")

<listOfCompartments>
	<compartment id="extracellular" constant="true"/>
	<compartment id="cytosol" constant="true"/>
</listOfCompartments>




## List of Parameters

In [259]:
f.write("<listOfParameters>\n\t<parameter id=\"main\" value=\"0.25\" constant=\"true\"/>\n</listOfParameters>\n\n")

## List of Qualitative Species (only if rdefba)

In [244]:
f.write("<qual:listOfQualitativeSpecies>\n")
for index, value in regulated['bool_ID'].items():
    f.write("\t<qual:qualitativeSpecies qual:id=\""+value+"\" qual:maxLevel=\"1\" qual:compartment=\"cytosol\" qual:constant=\"false\"/>\n")
f.write("</qual:listOfQualitativeSpecies>\n")

## List of Species

In [245]:
f.write("<listOfSpecies>\n")

16

### Metabolites

#### extracellular metabolites $\mathcal{Y}$

In [246]:
for index, value in extracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+extracellular['name'][index]+"\" compartment=\"extracellular\" initialAmount=\""+str(extracellular['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n")
    f.write("\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"extracellular\"/>\n\t\t</ram:RAM>\n\t</annotation>\n</species>\n")

#### extracellular metabolites $\mathcal{X}$

In [247]:
for index, value in intracellular['id'].items():
    f.write("<species id=\""+value+"\" name=\""+intracellular['name'][index]+"\" compartment=\"cytosol\" initialAmount=\"0.0\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n")
    f.write("\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:species ram:molecularWeight=\"0.0\" ram:objectiveWeight=\"0.0\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"metabolite\"/>\n\t\t</ram:RAM>\n\t</annotation>\n</species>\n")

### Macromolecules $\mathcal{P}$

#### Enzymes $\mathcal{E}$ (including Ribosome)

In [248]:
for index, value in enzymes['id'].items():
    f.write("<species id=\""+value+"\" name=\""+enzymes['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(enzymes['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n")
    f.write("\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:species ram:molecularWeight=\""+str(enzymes['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(enzymes['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n\t\t</ram:RAM>\n\t</annotation>\n</species>\n")

#### Regulatory Proteins $\mathcal{RP}$

In [249]:
for index, value in rp['id'].items():
    f.write("<species id=\""+value+"\" name=\""+rp['name'][index]+"\" compartment=\"cytosol\" initialAmount=\""+str(rp['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n")
    f.write("\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:species ram:molecularWeight=\""+str(rp['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(rp['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\"0.0\" ram:speciesType=\"enzyme\"/>\n\t\t</ram:RAM>\n\t</annotation>\n</species>\n")

#### Quota $\mathcal{Q}$

In [250]:
for index, value in quota['ID'].items():
    f.write("<species id=\""+value+"\" compartment=\"cytosol\" initialAmount=\""+str(quota['initialAmount'][index])+"\" constant=\"false\" boundaryCondition=\"false\" hasOnlySubstanceUnits=\"true\">\n")
    f.write("\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:species ram:molecularWeight=\""+str(quota['ram:molecularWeight'][index])+"\" ram:objectiveWeight=\""+str(quota['ram:objectiveWeight'][index])+"\" ram:biomassPercentage=\""+str(quota['ram:biomassPercentage'][index])+"\" ram:speciesType=\"quota\"/>\n\t\t</ram:RAM>\n\t</annotation>\n</species>\n")    

In [251]:
f.write("</listOfSpecies>\n\n")

18

## List of Reactions $\mathcal{R}$

In [252]:
f.write("<listOfReactions>\n")

18

### Metabolic Reactions $\mathcal{R_X}$ and $\mathcal{R_Y}$

In [253]:
stoich = 1.0
stoich_bool = False
for index, value in reactions_turnover['Covert ID'].items():
    f.write("<reaction id=\""+value+"\" reversible=\""+str(reactions_turnover['reversible'][index]).lower()+"\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\""+str(reactions_turnover['kcat_fwd/h'][index])+"\" ram:kcatBackward=\""+str(reactions_turnover['kcat_bwd/h'][index])+"\" ram:maintenanceScaling=\"0.0\"/>\n\t\t</ram:RAM>\n\t</annotation>\n")
    if pd.isna(reactions_turnover['Gene'][index]): # spontaneous reactions
        pass
    else:
        f.write("\t<fbc:geneProductAssociation fbc:id=\"E_"+value+"\">\n\t\t<fbc:geneProductRef fbc:geneProduct=\"E_"+value+"\"/>\n\t</fbc:geneProductAssociation>\n")
    f.write("\t<listOfReactants>\n")
    for g in re.split(r'[+\s]\s*', reactions_turnover['Reaction'][index]):
        if '-' in g:
            f.write("\t</listOfReactants>\n\t<listOfProducts>\n")
        else:
            try:
                stoich = float(g)
                stoich_bool = True
            except ValueError:
                if g == "":
                    pass
                else:
                    if stoich_bool == False:
                        stoich = 1.0
                    f.write("\t\t<speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                    stoich_bool = False
    f.write("\t</listOfProducts>\n</reaction>\n")

KeyError: 'kcat_fwd/h'

### Translation Reactions $\mathcal{R_P}$

In [ ]:
for index, value in translation['ID'].items():
    if any(c.islower() for c in value):
        identifier = 'RP_' + value[2:]
    else:
        identifier = 'E_' + value[2:]
    f.write("<reaction id=\""+value+"\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\""+str(translation['ram:kcatForward'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n\t\t</ram:RAM>\n\t</annotation>\n")    
    f.write("\t<fbc:geneProductAssociation fbc:id=\"E_R\">\n\t\t<fbc:geneProductRef fbc:geneProduct=\"E_R\"/>\n\t</fbc:geneProductAssociation>\n")
    f.write("\t<listOfReactants>\n\t\t<speciesReference species=\"Z\" stoichiometry=\""+str(translation['Length'][index])+"\" constant=\"true\"/>\n\t\t<speciesReference species=\"ATP\" stoichiometry=\""+str(translation['Length'][index]*4)+"\" constant=\"true\"/>\n\t</listOfReactants>\n")
    f.write("\t<listOfProducts>\n\t\t<speciesReference species=\""+identifier+"\" stoichiometry=\"1.0\" constant=\"true\"/>\n\t\t<speciesReference species=\"ADP\" stoichiometry=\""+str(translation['Length'][index]*4)+"\" constant=\"true\"/>\n\t\t<speciesReference species=\"PI\" stoichiometry=\""+str(translation['Length'][index]*4)+"\" constant=\"true\"/>\n\t</listOfProducts>\n</reaction>\n")
# 3 ATP: 1x ATP-AMT, 2x GTP-GDP (AMP und PPI sind im model!)

### Quota: nur für quota rest! quota protein is bei translation!
(if isnan(quota["gene"]))...

In [ ]:
stoich = 1.0
stoich_bool = False
for index, value in quota['ID'].items():
    f.write("<reaction id=\"Production_"+value+"\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\""+str(quota['kcat_fwd'][index])+"\" ram:kcatBackward=\"0.0\" ram:maintenanceScaling=\"0.0\"/>\n\t\t</ram:RAM>\n\t</annotation>\n")
    if pd.isna(quota['Gene'][index]): # spontaneous reactions
        pass
    else:
        f.write("\t<fbc:geneProductAssociation fbc:id=\"E_R\">\n\t\t<fbc:geneProductRef fbc:geneProduct=\"E_R\"/>\n\t</fbc:geneProductAssociation>\n")
    f.write("\t<listOfReactants>\n")
    for g in re.split(r'[+\s]\s*', quota['Reaction'][index]):
        if '-' in g:
            f.write("\t</listOfReactants>\n\t<listOfProducts>\n")
        else:
            try:
                stoich = float(g)
                stoich_bool = True
            except ValueError:
                if g == "":
                    pass
                else:
                    if stoich_bool == False:
                        stoich = 1.0
                    if g =='Biomass':
                        g = 'Quota_R'
                    f.write("\t\t<speciesReference species=\""+g+"\" stoichiometry=\""+str(stoich)+"\" constant=\"true\"/>\n")
                    stoich_bool = False
    f.write("\t</listOfProducts>\n</reaction>\n")

### Maintenance Reaction

In [ ]:
print("<reaction id=\"Maintenance\" name=\"Maintenance\" reversible=\"false\">\n\t<annotation>\n\t\t<ram:RAM xmlns:ram=\"https://www.fairdomhub.org/sops/304\">\n\t\t\t<ram:reaction ram:kcatForward=\"\" ram:kcatBackward=\"\" ram:maintenanceScaling=\"main\"/>\n\t\t</ram:RAM>\n\t</annotation>\n\t<listOfReactants>\n\t\t<speciesReference species=\"ATP\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfReactants>\n\t<listOfProducts>\n\t\t<speciesReference species=\"ADP\" stoichiometry=\"1\" constant=\"true\"/>\n\t\t<speciesReference species=\"PI\" stoichiometry=\"1\" constant=\"true\"/>\n\t</listOfProducts>\n</reaction>\n")

In [ ]:
f.write("</listOfReactions>\n\n")

## List of Gene Products

### Enzymes (including Ribosome)

In [ ]:
f.write("<fbc:listOfGeneProducts>\n")

In [ ]:
for index, value in enzymes['id'].items():
    f.write("\t<fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+enzymes['Gene'][index]+"_"+value[2:]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")
# Gene_Protein in order to have unique fbc:label values

### Regulatory Proteins

In [ ]:
for index, value in rp['id'].items():
    f.write("\t<fbc:geneProduct fbc:id=\""+value+"\" fbc:label=\""+rp['Gene'][index]+"\" fbc:associatedSpecies=\""+value+"\"/>\n")

In [ ]:
f.write("</fbc:listOfGeneProducts>\n\n")

## Last lines

In [ ]:
f.write("</model>\n")
f.write("</sbml>")

In [ ]:
f.close()